# Incremental deep learning with river & tf keras
## Imports

In [1]:
from pathlib import Path

from river import preprocessing, tree, evaluate, datasets, compose
import river

In [2]:
n_samples = 1_000
seed = 42
track_name = "RBF"

In [3]:
dataset = datasets.synth.RandomRBF(seed_model=7, seed_sample=seed,n_classes=10,n_features=200).take(n_samples)
metric = river.metrics.ROCAUC()

In [4]:
from IncrementalDL.OnlineTorch.classifier import PyTorch2RiverClassifier
from torch import nn, optim


def build_fn(n_features):
    net = nn.Sequential(
        nn.Linear(n_features, 5),
        nn.ReLU(),
        nn.Linear(5, 5),
        nn.ReLU(),
        nn.Linear(5, 5),
        nn.ReLU(),
        nn.Linear(5, 5),
        nn.ReLU(),
        nn.Linear(5, 1),
        nn.Softmax()
    )
    return net

In [5]:


model = compose.Pipeline(
    preprocessing.StandardScaler(),
    PyTorch2RiverClassifier(
                build_fn=build_fn,
                loss_fn=nn.BCELoss,
                optimizer_fn=optim.Adam,
                batch_size=1,
                learning_rate=1e-3,
    )
)

In [6]:
for x, y in dataset:
    y_pred = model.predict_proba_one(x)
    model.learn_one(x, y)
    metric.update(y, y_pred)
metric

/Users/kulbach/Documents/environments/IncrementalDL/lib/python3.9/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


ROCAUC: 0.666518